In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
P3 = Place(model, 'P3')

T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')
T3 = Transition(model, 'T3')
T4 = Transition(model, 'T4')

jobclass = ClosedClass(model, 'Class1', 8, P1, 0)

In [3]:
# T1 - Competes for tokens: requires 2 tokens from P1, produces 2 to P2
mode1 = T1.add_mode('Mode1')
T1.setDistribution(mode1, Exp(2))
T1.setEnablingConditions(mode1, jobclass, P1, 2)
T1.setFiringOutcome(mode1, jobclass, P2, 2)

# T2 - Competes for tokens: requires 3 tokens from P1, produces 3 to P3
mode2 = T2.add_mode('Mode2')
T2.setDistribution(mode2, Exp(1))
T2.setEnablingConditions(mode2, jobclass, P1, 3)
T2.setFiringOutcome(mode2, jobclass, P3, 3)

# T3 - Returns tokens: takes 1 from P2, returns 1 to P1
mode3 = T3.add_mode('Mode3')
T3.setDistribution(mode3, Exp(4))
T3.setEnablingConditions(mode3, jobclass, P2, 1)
T3.setFiringOutcome(mode3, jobclass, P1, 1)

# T4 - Returns tokens: takes 2 from P3, returns 2 to P1
mode4 = T4.add_mode('Mode4')
T4.setDistribution(mode4, Exp(2))
T4.setEnablingConditions(mode4, jobclass, P3, 2)
T4.setFiringOutcome(mode4, jobclass, P1, 2)

In [4]:
# Set up routing matrix
routingMatrix = model.init_routing_matrix()

# Place to transition routing (competing transitions from P1)
routingMatrix.set(jobclass, jobclass, P1, T1, 1.0)  # P1 -> T1
routingMatrix.set(jobclass, jobclass, P1, T2, 1.0)  # P1 -> T2 (competing)
routingMatrix.set(jobclass, jobclass, P2, T3, 1.0)  # P2 -> T3
routingMatrix.set(jobclass, jobclass, P3, T4, 1.0)  # P3 -> T4

# Transition to place routing
routingMatrix.set(jobclass, jobclass, T1, P2, 1.0)  # T1 -> P2
routingMatrix.set(jobclass, jobclass, T2, P3, 1.0)  # T2 -> P3
routingMatrix.set(jobclass, jobclass, T3, P1, 1.0)  # T3 -> P1
routingMatrix.set(jobclass, jobclass, T4, P1, 1.0)  # T4 -> P1

model.link(routingMatrix)

In [5]:
# Set initial state - all 8 tokens start in P1
P1.setState([jobclass.getPopulation()])
P2.setState([0])
P3.setState([0])

In [6]:
# Solve the model
solver = SolverJMT(model, seed=23000)
avgTable = solver.get_avg_table()

JMT Model: /tmp/workspace/jsim/14206373839422455609/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.376436s.
  Station JobClass    QLen  Util   RespT  ResidT    ArvR    Tput
0      P1   Class1  3.4344   0.0  0.7203  0.7203  4.7960  4.7866
1      P2   Class1  2.5589   0.0  0.8665  0.8665  2.9984  3.0006
2      P3   Class1  1.9935   0.0  1.1405  1.1405  1.8044  1.7420
